In [1]:
import numpy as np
import pandas as pd
from nutshell import ModelData, Learner

Using TensorFlow backend.


In [2]:
# Fathom concepts

# FactSet is a dataset with three columns: subject, fact, value
# You can generate a FactSet from a data with rows and columns using 
#  source csv has one row per subject - with columns containing fact values
#  destination dataset has one row per subject/fact



In [59]:
class FactSet:
    
    def __init__(self):
        fact_data = pd.DataFrame()   # internal dataset with one row per subject/fact
        label_data = pd.DataFrame()  # internal dataset with one row per subject
        model_name = 'factset'       # override this value to the name of the model file that will be built
        model = None
        subject_column = ''          # name of subject id column
        fact_name_column = ''
        fact_value_column = ''
        label_column = ''            # name label column - this will be not be used in training
        fact_colnames = []           # optional: list of fact columns; default is all non subject/label columns
               
    def load_subject_rows(self, df):
        
        # dataset is in the format one row per subject, one fact per column
        print('Transposing data into fact rows...')
        
        #to do
        
    
    def load_fact_rows(self, df):
               
        # dataset is in the format one row per subject/fact, each row has a fact name and value column
        self.fact_data = pd.DataFrame()
        self.fact_data['subject'] = df[self.subject_column]
        self.fact_data['fact_name'] = df[self.fact_name_column]
        self.fact_data['fact_value'] = df[self.fact_value_column]
        
        print(len(df), 'fact rows loaded')
    
    def learn(self, model_name=''):
     
        # training a neural net to tell which facts are true and which are false about each subject
        #  so for every true fact about a subject, there should be an equal number of false facts
        #  false facts are taken from other subjects, so they are plausible facts
    
        if model_name == '':
            model_name = self.model_name
    
        print('Preparing data for model training...')

        self.fact_data['is_true'] = 1
        
        data = ModelData(self.fact_data)
        data.category_columns = ['subject','fact_name']
        data.label_column = 'is_true'
        data.prepare_data()
        

        print('Building neural network...')
        
        print('Training neural network...')
        
        print ('Stored model to: ', model_name)
        
    
        
    



In [37]:
# as a sample, use instacart purchases
# we will use users as subjects and products purchased as facts

dataPath = '/Users/Miles/Documents/Datasets/Instacart/'
dfProducts = pd.read_csv(dataPath + 'products.csv')
dfOrders = pd.read_csv(dataPath + 'orders.csv')
dfOrderProducts = pd.read_csv(dataPath + 'order_products__prior.csv')

In [39]:
# full set has 206K unique users
dfOrders['user_id'].nunique()

# lets start with a sample set of 2000 users
dfSample = pd.DataFrame()
dfSample['user_id'] = dfOrders['user_id'].unique()
dfSample = dfSample.sample(n=2000)

dfOrders = pd.merge(dfSample, dfOrders, on='user_id').reset_index()

In [41]:
# create list of user ids and products they have ordered

dfUserProducts = pd.merge(dfOrders, dfOrderProducts, on='order_id')[['user_id', 'product_id']].\
    sort_values(['user_id', 'product_id']).groupby(['user_id', 'product_id']).count().reset_index()

In [49]:
dfUserProducts['fact_name'] = 'product'

In [56]:
facts = FactSet()
facts.subject_column = 'user_id'
facts.fact_name_column = 'fact_name'
facts.fact_value_column = 'product_id'
facts.load_fact_rows(dfUserProducts)

129146 fact rows loaded


In [48]:
print(dfUserProducts.columns)

Index(['user_id', 'product_id'], dtype='object')


In [13]:
dfOrderProducts[0:5]

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
